In [8]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd


In [9]:
model = gp.Model("VRPLTT")

In [10]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [11]:
load_levels

array([ 20.,  60., 100., 140.])

In [12]:
upper

array([ 40.,  80., 120., 160.])

In [16]:
def vel(m, h, P , P_electric = 250):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eta = 0.95
        
    c0 = P*eta
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))
    c3 = c_Fd
    
    v_max = 30 #km/h
   ### closed form solution of 3-rd degree polynomial

    # p = c1/c3
    # q = -c0/c3
    # print(np.roots([1, 0, p, q]))
    
    ###
    coefs = [c3, 0, c1, -eta*P]
    coefs_electric = [c3, 0, c1, -eta*(P+P_electric)]        
    v = np.max(np.real(3.6*np.roots(coefs)))
    v_electric = np.max(np.real(3.6*np.roots(coefs_electric)))
    if v > 25:
        #print('up-cyclist only')
        return min(v, v_max)
    else:
        #print('up-electric')
        return min(v_electric, 25)


def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    #print(vel(m,h,P))
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P, P_electric=250)
    return np.round(t_ij_mat,4)

The forces acting on the bicyle are: the aerodynamic drag resistance $F_{D}$, the rolling resistance $F_{R}$ and the component of the gravity force along the direction of motion $F_{G}$:

* $ F_{D} = \frac{\rho C_{D}Av^{2}}{2}$
* $ F_{R} = C_{R}mg cos(arctan(h))$
* $ F_{G} = mg sin(arctan(h))$

We have to distinguish two cases: uphill and downhill.
In uphill, the component of the gravity force along the direction of motion is a resistance force, while in downhill it helps the motion of the bicyle.
so we have, at equilibrium:

* $\mathrm{ \eta P_{traction} = (F_{D} + (F_{R} + F_{G})v }$

where $\eta$ is the efficiency of the mechanic transmission, which is assumed to be 0.95.

If we use a negative h in the formuals above when traversing an edge with negative slope, since $cos(\alpha) = cos(-\alpha)$ and $sin(\alpha) = -sin(-\alpha)$, we have just one expression for the equation to solve.

Moreover, we assume that the bicyle speed is limited in downhill for safety reasons. Within cities, a typical speed limit for for motor vehicles is 50km/h. Considering also that the regulation for pedelec bicyle allows a maximum speed of 25km/h in electric assisted mode, we set a maximum speed in downhill of 30km/h.
The delta energy can be used for recharging the battery for example, similarly to other solutions in the automaotive industry.
The maximum speed limit is imposed on uphill as well.
The strategy for solving the equation is:
* solve the equation for the two cases: (1) $P=P_{cyclist}$, and (2) $P=P_{cyclist}+P_{electric}$ 
* If the cyclist can reach a speed that is higher than the speed allowed in electric mode, use that speed, subject to the limit of maximum speed allowed,
* otherwise use the minimum value among 25 km/h and the solution of problem (2)

In practice we have $P_{electric}$ = 250W, while for $\overline {P_{cyclist}}$ we assume to have few different types of cyclist.

In [19]:
df = pd.read_csv("./instances/small/Fukuoka_01.csv")
P_ciclista = 150 # Watt

t=time_matrix(df,load_levels,P=P_ciclista)

In [23]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 0.456,
 2: 1.8144,
 3: 3.1721,
 4: 4.1537,
 5: 4.1697,
 6: 4.5137,
 7: 6.0537,
 8: 5.9869,
 9: 8.3614,
 10: 7.7014000000000005,
 11: 7.1433,
 12: 4.0309,
 13: 3.8477,
 14: 5.3289,
 15: 7.501200000000001,
 16: 7.138199999999999,
 17: 2.8697,
 18: 2.4377,
 19: 1.3866,
 20: 2.6727}
{0: None,
 1: 0,
 2: 1,
 3: 2,
 4: 18,
 5: 18,
 6: 18,
 7: 13,
 8: 12,
 9: 10,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 12,
 15: 1,
 16: 14,
 17: 18,
 18: 0,
 19: 0,
 20: 19}


In [31]:
for i in t[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	0.456 	2.049 	4.208 	5.734 	6.899 	8.16 	10.234 	11.427 	11.768 	8.897 	8.338 	5.235 	4.504 	7.498 	8.34 	9.217 	3.095 	2.438 	1.387 	2.769 

	0.456 	0.0 	1.358 	2.876 	4.333 	5.558 	6.893 	8.836 	9.786 	10.12 	7.245 	6.687 	3.575 	3.392 	6.787 	7.045 	8.765 	3.865 	3.561 	1.91 	2.443 

	0.696 	0.36 	0.0 	1.358 	2.797 	4.057 	5.446 	7.326 	8.418 	9.381 	6.802 	5.888 	3.14 	2.948 	6.344 	6.602 	8.322 	3.432 	3.378 	0.989 	2.812 

	1.01 	0.674 	0.314 	0.0 	1.439 	2.68 	3.916 	5.964 	7.16 	8.121 	6.915 	4.627 	3.392 	3.93 	6.923 	7.045 	8.738 	1.488 	4.566 	1.138 	0.988 

	1.342 	1.006 	0.646 	0.332 	0.0 	0.3 	0.646 	4.498 	6.187 	8.138 	6.932 	4.644 	4.835 	5.366 	8.352 	8.488 	10.178 	1.82 	1.716 	1.47 	1.32 

	1.642 	1.306 	0.946 	0.632 	1.229 	0.0 	0.344 	3.255 	4.886 	7.026 	5.906 	5.366 	6.138 	6.662 	9.644 	8.088 	9.971 	2.122 	3.554 	1.772 	1.62 

	1.988 	1.65 	1.292 	0.978 	2.499 	1.158 	0.0 	1.78 	3.383 	5.714 	5.689 	5.87 	7.63 	8.143 	9.955 	7.871 	9.754 	2.466 	4.878 	2

In [26]:
%%timeit
dist,parent = dijkstra(0,t)


405 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [30]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix



In [29]:

sp = shortest_path_matrix(t)
for i,ii in (sp[:,:,0]:
    for j in i:
        print(f"\t{round(j,2)}",end=" ")
    print("\n")
    

ValueError: too many values to unpack (expected 2)